In [1]:
import sys
import pandas as pd
import math
import os
import argparse
import numpy as np

In [2]:
#Function to parse .ms file data into nested list of positions and genotypes
#Function takes as input open reading file object and no. of samples
def get_nested_data_list(inPath, pop, model, rep, chr_len=198345):
    pops = {'AFR':'p1', 'EUR':'p3', 'EAS':'p5', 'SAS':'p4'}
    samples = {"AFR": 198, "EUR": 1004, "EAS": 208, "SAS": 978}
    l_Pos = [] #list of positions of SNPs
    l_Genos = [] #list of alleles
    d_tmp = {} #dict to store individual allele info for each individual (values) at each site (keys)
    
    f_ms = open(inPath + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".ms", 'r')
    #positions on line 2
    pos_lines = [2]
    for position, line in enumerate(f_ms):
        if position in pos_lines:
            #store positions in list
            pos_list  = line.split()
    #Set file pointer to start of file
    f_ms.seek(0)

    i = 0
    #Loop through positions, storing in list
    for pos in pos_list[1:]:
        #Append position to l_Pos (after converting to float)
        x = int(np.round(float(pos)*chr_len,0))
        l_Pos.append(int(x))
        #Add dictionary key for each position, with empty value
        d_tmp[str(i)] = ""
        i += 1  

    #genotypes on line 3 onwards (use samples argument to determine length of file)
    g_lines = [x for x in range(3, samples[pop] + 4)]
    #Loop through lines (ie individuals)
    for position, line in enumerate(f_ms):
        if position in g_lines:
            #Remove newline character
            line1 = line.strip('\n')
            i = 0
            #For each individual, loop through each site, appending allele information for that individual to 
            #the site number in dict
            while i < len(line1):
                d_tmp[str(i)] = d_tmp[str(i)] + line1[i]
                i = i + 1

    f_ms.seek(0)

    #Create nested list of positions and genotypes
    l_data = [[j, d_tmp[str(i)]] for i,j in enumerate(l_Pos)]
    
    #Sum genotype information to get DACs
    for i,j in enumerate(l_data):
        l_data[i].append(sum([int(x) for x in l_data[i][1]]))
    
    #Convert dictionary to dataframe, removing genotype column
    df = pd.DataFrame(l_data)[[0,2]]
    #Rename columns
    df.columns = ['position', 'x']
    #Add samples info
    df['n'] = samples[pop]
    #Add fold info
    df['folded'] = 0

    #Read in .fixed file
    fixed = pd.read_csv(inPath + "/rep" + str(rep) + "_model" + str(model) + ".fixed", skiprows=2, sep=' ',
                   names=['tempID', 'permID', 'mutType', 'position', 's', 'h', 'initial_subpop', 'originTick',
                         'fix_gen'])
    fixed[(fixed.originTick>=94000)]
    fixed = fixed[['position']]
    
    f = pd.read_csv(inPath + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".fixed", sep=' ', 
                names=['OUT', 'tick', 'cycle', 'T', 'subpopID', 'mutID', 'mutType', 'position', 's', 'h', 'originPop', 'originTick', 'AC'])

    #Subset for only relevant population and mutations that occur after burnin
    f = f[(f.subpopID == pops[pop])& (f.originTick>=94000)]
    #Keep only position column
    f = f[['position']]

    f = pd.concat([fixed,f])
    f = f.drop_duplicates()

    #Create dataframe of fixed mutations
    f_df = pd.DataFrame([[x,samples[pop],samples[pop],0] for x in f.position])
    if(len(f_df)>0):
        f_df.columns = df.columns
        #Concatenate dfs and sort by position
        aff = pd.concat([df, f_df]).sort_values('position').reset_index(drop='True')
    else:
        aff = df.reset_index(drop='True')
    #Remove duplicates, keeping the higher value (ie in case the beneficial has overlayed a previous mutation)
    aff = aff.groupby('position', group_keys=False).apply(lambda x: x.loc[x['x'].idxmax()], include_groups=False)
    aff['position'] = aff.index
    aff = aff.reset_index(drop=True)
    aff = aff[['position','x','n','folded']]
    
    return(aff)

In [3]:
model = 1
for Nes in [1000, 10000]:
    for pop in ['AFR', 'EUR', 'EAS', 'SAS']:
        for rep in range(1, 101):
            df = get_nested_data_list(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/sims/" + str(Nes), pop, model, rep, chr_len=198345)
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".aff", 
                      sep='\t', header=True, index=False)
            df = df[['position']]
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".grid", 
                      sep='\t', header=False, index=False)

model = 2
for Nes in [1000, 10000]:
    for pop in ['EUR', 'EAS', 'SAS']:
        for rep in range(1, 101):
            df = get_nested_data_list(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/sims/" + str(Nes), pop, model, rep, chr_len=198345)
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".aff", 
                      sep='\t', header=True, index=False)
            df = df[['position']]
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".grid", 
                      sep='\t', header=False, index=False)
            
model = 3
for Nes in [1000, 10000]:
    for pop in ['EUR']:
        for rep in range(1, 101):
            df = get_nested_data_list(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/sims/" + str(Nes), pop, model, rep, chr_len=198345)
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".aff", 
                      sep='\t', header=True, index=False)
            df = df[['position']]
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".grid", 
                      sep='\t', header=False, index=False)

In [6]:
model = 3
for Nes in [1000, 10000]:
    for pop in ['EUR']:
        for rep in range(1, 101):
            f_aff = r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".aff"
            f_rec = r"/home/vsoni11/human_demog_DFE/rr_maps/" + str(rep) + ".txt"
            #Read in aff file
            df = pd.read_csv(f_aff, header=0, sep='\t')
            #Read in rr file
            rr = pd.read_csv(f_rec, names=['pos','ro'], sep='\t')
            
            #Create nested list of ro at each position
            l = [[x*10**-6] * 10000 for x in rr.ro]
            #Flattern nested list
            lst = [item for sublist in l for item in sublist]
            
            #Create list for results, starting with a 0
            res = [0]
            #Loop through positions
            for i in range(0, len(df.position)-1):
                #Set indexes to calculate sum of rates between polymorphisms
                p2 = df.position[i]
                p1 = df.position[i+1] 
                res.append(np.sum(lst[p2:p1]))
            
            df['rate'] = res
            df = df[['position', 'rate']]
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".rec", 
                      sep='\t', header=True, index=False)

In [7]:
for Nes in [100, 1000, 10000]:
    for pop in ['AFR', 'EUR', 'EAS', 'SAS']:
        for model in range(1, 4):
            df = pd.DataFrame()
            for rep in range(1, 101):
                f_aff = r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_rep" + str(rep) + "_model" + str(model) + ".aff"
                fdf = pd.read_csv(f_aff, header=0, sep='\t')
                df = pd.concat([df, fdf])
            df.to_csv(r"/ospool/ap21/data/vsoni11/hdd_res/sweep_detection/singleSweep/SF_files/" + str(Nes) + "/" + pop + "_model" + str(model) + "_all.aff", 
                      sep='\t', header=True, index=False)

In [10]:
m = 1
res = []
pops = ['AFR', 'EUR', 'EAS', 'SAS']
for Nes in [1000, 10000]:
    for p in pops:
        for i in range(1, 101):
            tmp = [Nes,p,m,i]
            res.append(tmp)

df = pd.DataFrame(res)
df.to_csv(r"/home/vsoni11/SF_realisations.txt", header=False, index=False, sep='\t')

m = 2
res = []
pops = ['EUR', 'EAS', 'SAS']
for Nes in [1000, 10000]:
    for p in pops:
        for i in range(1, 101):
            tmp = [Nes,p,m,i]
            res.append(tmp)

df = pd.DataFrame(res)
df.to_csv(r"/home/vsoni11/SF_realisations.txt", header=False, index=False, sep='\t', mode='a')

m = 3
res = []
pops = ['EUR']
for Nes in [1000, 10000]:
    for p in pops:
        for i in range(1, 101):
            tmp = [Nes,p,m,i]
            res.append(tmp)

df = pd.DataFrame(res)
df.to_csv(r"/home/vsoni11/SF_realisations.txt", header=False, index=False, sep='\t', mode='a')